# 패키지 다운로드

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pylab as plt

# 파일불러오기

In [3]:
data1_2020 = pd.read_excel("1. (raw) 사고 발생 데이터_수정.xlsx", sheet_name=2) #2020년 불러오기
data1_2021 = pd.read_excel("1. (raw) 사고 발생 데이터_수정.xlsx", sheet_name=3) #2021년 불러오기
data1_2022 = pd.read_excel("1. (raw) 사고 발생 데이터_수정.xlsx", sheet_name=4) #2022년 불러오기

In [4]:
data=pd.concat([data1_2020,data1_2021,data1_2022], ignore_index = True)
#2020~2022년까지 concat 함수를 이용하여 합치기

In [5]:
data.head()

,구분,학교명,지역,교육청,설립유형,학교급,사고자구분,사고자성별,사고자학년,사고접수일,사고발생일,사고발생요일,사고발생시간,사고시간,사고장소,사고부위,사고형태,사고매개물,사고당시활동
0,A0000001,치악초등학교,강원,원주교육지원청,공립,초,일반학생,여,5학년,2020-01-10,2018-11-01,목,16:00,특별활동,교실,다리,기타,기타,실험실습
1,A0000002,정선초등학교,강원,정선교육지원청,공립,초,일반학생,여,6학년,2020-01-22,2019-06-14,금,14:40,체육수업,운동장,다리,낙상-넘어짐,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),구기운동
2,A0000003,정선초등학교,강원,정선교육지원청,공립,초,일반학생,남,4학년,2020-01-22,2019-06-20,목,11:30,학교행사,교외활동,다리,낙상-넘어짐,기타,기타운동
3,A0000004,태장초등학교,강원,원주교육지원청,공립,초,일반학생,남,6학년,2020-06-19,2019-08-30,금,13:50,체육수업,부속시설,손,물리적힘 노출,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),구기운동
4,A0000005,버들중학교,강원,원주교육지원청,공립,중,일반학생,여,2학년,2020-02-07,2019-10-08,화,09:40,학교행사,운동장,다리,낙상-미끄러짐,건물(문/창문/바닥/벽 등),장난/놀이


# 전처리

## 사고발생시간 전처리

In [6]:
def remove_minute(input_string):
    input_string = input_string.split(":") # ':'을 기준으로 시간중 시만 살리기
    modified_string = input_string[0]
    return modified_string

In [7]:
#'new_사고발생시간' 컬럼 추가
data['new_사고발생시간'] = data['사고발생시간'].apply(remove_minute)

In [8]:
def remove_minute1(input_float):
    input_float = int(input_float)
    if 0<= input_float <6:
        modified_string = "새벽" #0시~6시 사이 = 새벽
    elif 6<= input_float <12:
        modified_string = "아침" #6시 ~12시 사이 = 아침
    elif 12 <= input_float <16:
        modified_string = "낮" #12시~16시 사이 = 낮
    elif input_float >=16:
        modified_string = "저녁" #16이후는 저녁
    return modified_string

In [9]:
#'new_시간대' 컬럼 추가
data['new_시간대'] = data['new_사고발생시간'].apply(remove_minute1)

## 사고자학년 전처리

In [10]:
data = data.dropna() #결측지 제거

In [11]:
def remove_string(input_string):
    if len(input_string) > 2:
        modified_string = input_string[:-2]  # 뒤에서 3번째까지의 문자를 제거
        return modified_string
    else:
        # 문자열의 길이가 3보다 작을 경우, 그대로 반환
        return input_string

In [12]:
#'new_사고자학년' 컬럼 추가
data['new_사고자학년'] = data['사고자학년'].apply(remove_string)

## 사고발생 연, 월, 일 컬럼 생성

In [13]:
#to_dataetime 함수 이용해 문자열을 datetime 형식으로 변환하거나 리스트, 배열, 데이터프레임의 열을 변환
#'사고발생일' 컬럼 추가
data['사고발생일'] = pd.to_datetime(data['사고발생일'], format='%Y-%m-%d')

In [14]:
# '사고발생연도', '사고발생월', '사고발생일자' 컬럼 추가
data['사고발생연도'] = data['사고발생일'].dt.year #년도
data['사고발생월'] = data['사고발생일'].dt.month #월
data['사고발생일자'] = data['사고발생일'].dt.day #일

## 사고 부위(머리, 상체, 하체, 기타)
- 머리(두부) + 치아(구강) -> 머리
- 손 + 팔 + 흉복부 -> 상체
- 다리 + 발 -> 하체
- 기타 + 복합부위 -> 기타

In [15]:
mapping = {
    '머리(두부)': '머리',
    '치아(구강)': '머리',
    '손': '상체',
    '팔': '상체',
    '흉복부': '상체',
    '다리': '하체',
    '발': '하체',
    '기타': '기타',
    '복합부위': '기타'
}

#'new_사고부위' 컬럼 추가
data['new_사고부위'] = data['사고부위'].replace(mapping)

## 학교급 전처리

- 초등학교, 초 - > '초'로 통일
- 중학교, 중 -> '중'으로 통일
- 고등학교, 고 -> '고'로 통일
- 유치원, 유 -> '유'로 통일
- 특수학교, 기타학교, 특수, 기타, 복합부위 -> '기타'로 통일

In [16]:
mapping = {
    '초등학교': '초',
    '중학교': '중',
    '고등학교': '고',
    '유치원': '유',
    '초': '초',
    '중': '중',
    '고': '고',
    '유': '유',
    '특수학교': '기타',
    '기타학교': '기타',
    '특수': '기타',
    '기타': '기타',
    '유': '유',
    '복합부위': '기타'
}

#'new_학교급' 컬럼 추가
data['new_학교급'] = data['학교급'].replace(mapping)

## drop

In [17]:
#불필요한 컬럼 지우기
data = data.drop(["구분", "학교명", "교육청", "설립유형"
                ,"사고접수일", "사고발생시간","사고자학년"], axis =1)

In [18]:
#데이터 확인
data

,지역,학교급,사고자구분,사고자성별,사고발생일,사고발생요일,사고시간,사고장소,사고부위,사고형태,사고매개물,사고당시활동,new_사고발생시간,new_시간대,new_사고자학년,사고발생연도,사고발생월,사고발생일자,new_사고부위,new_학교급
0,강원,초,일반학생,여,2018-11-01,목,특별활동,교실,다리,기타,기타,실험실습,16,저녁,5,2018,11,1,하체,초
1,강원,초,일반학생,여,2019-06-14,금,체육수업,운동장,다리,낙상-넘어짐,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),구기운동,14,낮,6,2019,6,14,하체,초
2,강원,초,일반학생,남,2019-06-20,목,학교행사,교외활동,다리,낙상-넘어짐,기타,기타운동,11,아침,4,2019,6,20,하체,초
3,강원,초,일반학생,남,2019-08-30,금,체육수업,부속시설,손,물리적힘 노출,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),구기운동,13,낮,6,2019,8,30,상체,초
4,강원,중,일반학생,여,2019-10-08,화,학교행사,운동장,다리,낙상-미끄러짐,건물(문/창문/바닥/벽 등),장난/놀이,09,아침,2,2019,10,8,하체,중
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284421,경기,고등학교,일반학생,남,2022-12-29,목,체육수업,부속시설,발,낙상-넘어짐,자연(사람/동물/식물 등),구기운동,12,낮,2,2022,12,29,하체,고
284422,경기,중학교,일반학생,남,2022-12-30,금,학교행사,운동장,손,물리적힘 노출,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),기타,12,낮,1,2022,12,30,상체,중
284423,경기,중학교,일반학생,남,2022-12-30,금,점심시간,운동장,발,낙상-미끄러짐,건물(문/창문/바닥/벽 등),보행/주행,13,낮,1,2022,12,30,하체,중
284424,경기,중학교,일반학생,남,2022-12-30,금,등하교,운동장,팔,낙상-미끄러짐,건물(문/창문/바닥/벽 등),보행/주행,09,아침,1,2022,12,30,상체,중


In [19]:
data["지역"].unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

- 특별시, 광역시, 특별자치시 > 해당 광역 해정 구역으로 포함

In [20]:
mapping = {
    "경기" : "경기도",
    "인천" : "경기도",
    "세종" : "충청남도",
    "경남" : "경상남도",
    "경북" : "경상북도",
    "광주" : "전라남도",
    "전남" : "전라남도",
    "전북" : "전라북도",
    "대구" : "경상북도",
    "부산" : "경상남도",
    "경남" : "경상남도",
    "경북" : "경상북도", 
    "울산" : "경상북도",
    "충남" : "충청남도",
    "충북" : "충청북도",
    "대전" : "충청북도",
    "제주" : "제주도",
    "강원" : "강원도"
}

data['지역'] = data['지역'].replace(mapping)

In [21]:
data["지역"].unique()

array(['강원도', '경기도', '경상남도', '경상북도', '전라남도', '충청북도', '서울', '충청남도', '전라북도',
       '제주도'], dtype=object)

# 지역별 기사청 데이터

## 기온

In [22]:
import glob

# 불러올 CSV 파일들의 경로 패턴을 지정
file_pattern = '기온/*.csv'  # 폴더경로에는 실제 파일이 들어있는 디렉토리 경로를 입력하세요

# 해당 패턴에 매칭되는 모든 파일의 경로를 리스트로 가져옴
file_paths = glob.glob(file_pattern)

# 빈 리스트를 생성하여 데이터프레임들을 저장할 준비를 함
dataframes = []

# 파일들을 순회하면서 데이터프레임으로 읽어오기
for file_path in file_paths:
    # CSV 파일을 데이터프레임으로 읽어와서 리스트에 추가
    df = pd.read_csv(file_path, skiprows = 11,encoding = "cp949")
    dataframes.append(df)

# dataframes
# 모든 데이터프레임을 병합 (concatenate)
merged_temp = pd.concat(dataframes, ignore_index=True)

# 병합된 데이터프레임 출력 또는 활용
# merged_temp


In [23]:
merged_temp = merged_temp.iloc[:,1:4]
merged_temp = merged_temp.dropna(how = "all")

In [24]:
temp1 = merged_temp[merged_temp["지역명"].isin(["강원도","경기도","서울"])]
temp2 = merged_temp[~merged_temp["지역명"].isin(["강원도","경기도","서울"])]

## 강수량

In [25]:
import glob

# 불러올 CSV 파일들의 경로 패턴을 지정
file_pattern = '강수량/*.csv'  # 폴더경로에는 실제 파일이 들어있는 디렉토리 경로

# 해당 패턴에 매칭되는 모든 파일의 경로를 리스트로 가져옴
file_paths = glob.glob(file_pattern)

# 빈 리스트를 생성하여 데이터프레임들을 저장할 준비를 함
dataframes = []

# 파일들을 순회하면서 데이터프레임으로 읽어오기
for file_path in file_paths:
    # CSV 파일을 데이터프레임으로 읽어와서 리스트에 추가
    df = pd.read_csv(file_path,skiprows = 12, encoding = "cp949")
    dataframes.append(df)

# dataframes
# 모든 데이터프레임을 병합 (concatenate)
merged_rain = pd.concat(dataframes, ignore_index=True)

# 병합된 데이터프레임 출력 또는 활용
# merged_rain
        

In [26]:
merged_rain = merged_rain.iloc[:,1:4]
merged_rain = merged_rain.dropna(how = "all") #NaN값 지우기

- 년도, 월 분리를 위해 구성이 다른 지역들 별 데이터 프레임 제작

In [27]:
rain1 = merged_rain[merged_rain["지역명"].isin(["전남","전북","제주"])] #전남,전북,제주포함하는 데이터프레임
rain2 = merged_rain[~merged_rain["지역명"].isin(["전남","전북","제주"])] #전남,전북,제주를 포함하지 않는 데이터 프레임

## 풍속

In [28]:
import glob

# 불러올 CSV 파일들의 경로 패턴을 지정
file_pattern = '풍속/*.csv'  # 폴더경로에는 실제 파일이 들어있는 디렉토리 경로를 입력하세요

# 해당 패턴에 매칭되는 모든 파일의 경로를 리스트로 가져옴
file_paths = glob.glob(file_pattern)

# 빈 리스트를 생성하여 데이터프레임들을 저장할 준비를 함
dataframes = []

# 파일들을 순회하면서 데이터프레임으로 읽어오기
for file_path in file_paths:
    # CSV 파일을 데이터프레임으로 읽어와서 리스트에 추가
    df = pd.read_csv(file_path,skiprows = 13, encoding = "cp949")
    dataframes.append(df)

# dataframes
# 모든 데이터프레임을 병합 (concatenate)
merged_wind = pd.concat(dataframes, ignore_index=True)

# 병합된 데이터프레임 출력 또는 활용
# merged_wind
        

In [29]:
merged_wind = merged_wind.iloc[:,1:4]
merged_wind = merged_wind.dropna(how = "all") #NaN값 지우기

In [30]:
#년도 - 월 분리 중 년도 데이터
def remove_year(input_string):
    input_string = input_string.split("-") #'-' 을 기준으로 날짜 중 년도 데이터만 저장
    modified_string = input_string[0]
    return modified_string

#년도 - 월 분리 중 월 데이터
def remove_month(input_string):
    input_string = input_string.split("-") #'-'을 기준으로 날짜 중 월 데이터만 저장
    modified_string = input_string[1]
    return modified_string

In [31]:
#년도 - 월 분리 중 년도 데이터
def remove_year1(input_string):
    input_string = input_string.split("-")  #'-'을 기준으로 날짜 중 년도 데이터만 저장
    modified_string = input_string[1]
    return modified_string

#년도 - 월 분리 중 월 데이터
def remove_month1(input_string):
    input_string = input_string.split("-") #'-'을 기준으로 날짜 중 월 데이터만 저장
    modified_string = input_string[0]
    return modified_string

In [32]:
#기온에'월', '년도' 컬럼 추가
temp1['월'] = temp1['일시'].apply(remove_month)
temp1["년도"] = temp1['일시'].apply(remove_year)
temp2['월'] = temp2['일시'].apply(remove_month1)
temp2["년도"] = temp2['일시'].apply(remove_year1)

C:\Users\win\AppData\Local\Temp\ipykernel_10140\3539167549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1['월'] = temp1['일시'].apply(remove_month)
C:\Users\win\AppData\Local\Temp\ipykernel_10140\3539167549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1["년도"] = temp1['일시'].apply(remove_year)
C:\Users\win\AppData\Local\Temp\ipykernel_10140\3539167549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [33]:
temp = pd.concat([temp1, temp2]) #기온 데이터 합치기

In [34]:
#강수량에 '월','년도' 컬럼 추가
rain1['월'] = rain1['일시'].apply(remove_month)
rain1["년도"] = rain1['일시'].apply(remove_year)
rain2['월'] = rain2['일시'].apply(remove_month1)
rain2["년도"] = rain2['일시'].apply(remove_year1)

C:\Users\win\AppData\Local\Temp\ipykernel_10140\493735158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain1['월'] = rain1['일시'].apply(remove_month)
C:\Users\win\AppData\Local\Temp\ipykernel_10140\493735158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain1["년도"] = rain1['일시'].apply(remove_year)
C:\Users\win\AppData\Local\Temp\ipykernel_10140\493735158.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [35]:
rain = pd.concat([rain1, rain2]) #강수량 데이터 합치기

In [36]:
#평균풍속에 '월','년도' 컬럼 추가
merged_wind['월'] = merged_wind['일시'].apply(remove_month1)
merged_wind["년도"] = merged_wind['일시'].apply(remove_year1)
wind = merged_wind

- 기상청 데이터와 본데이터 지역명, 날짜 구성 맞추기

In [37]:
mapping = {
    "경기" : "경기도",
    "인천" : "경기도",
    "세종" : "경기도",
    "경남" : "경상남도",
    "경북" : "경상북도",
    "광주" : "전라남도",
    "전남" : "전라남도",
    "전북" : "전라북도",
    "대구" : "경상북도",
    "부산" : "경상남도",
    "경남" : "경상남도",
    "경북" : "경상북도", 
    "울산" : "경상북도",
    "충남" : "충청남도",
    "충북" : "충청북도",
    "대전" : "충청북도",
    "제주" : "제주도",
    "강원" : "강원도"
}

temp["지역명"] = temp["지역명"].replace(mapping)
rain["지역명"] = rain["지역명"].replace(mapping)
wind["지역명"] = wind["지역명"].replace(mapping)

In [80]:
mapping = {
    "Jan" : 1,
    "Feb" : 2,
    "Mar" : 3,
    "Apr" : 4,
    "May" : 5,
    "Jun" : 6,
    "Jul" : 7,
    "Aug" : 8,
    "Sep" : 9,
    "Oct" : 10,
    "Nov" : 11,
    "Dec" : 12,

}

temp["월"] = temp["월"].replace(mapping)
rain["월"] = rain["월"].replace(mapping)
wind["월"] = wind["월"].replace(mapping)

In [40]:
mapping = {
    "20" : 2020,
    "21" : 2021,
    "22" : 2022,
    20 : 2020,
    21 : 2012,
    22 : 2022

}

temp["년도"] = temp["년도"].replace(mapping)
rain["년도"] = rain["년도"].replace(mapping)
wind["년도"] = wind["년도"].replace(mapping)

In [41]:
#'지역명','년도','월' 기준으로 데이터 재정렬
temp = temp.sort_values(by=["지역명","년도","월"])
rain = rain.sort_values(by=["지역명","년도","월"])
wind = wind.sort_values(by=["지역명","년도","월"])

In [42]:
#인덱스 재설정
temp = temp.reset_index()
rain = rain.reset_index()
wind = wind.reset_index()

In [43]:
#불필요한 컬럼제거
rain = rain.drop(["일시","index"],axis =1)
temp = temp.drop(["일시","index"], axis =1)
wind = wind.drop(["일시","index"], axis = 1)

#데이터 컬럼명 재설정
rain = rain.rename(columns={'평균월강수량(mm)':'강수량'})
temp = temp.rename(columns={'평균기온(℃)':'평균기온'})
wind = wind.rename(columns={'평균풍속(m/s)':'평균풍속'})

- 데이터 int타입으로 바꾸기

In [44]:
rain["월"] = rain["월"].astype(int)
rain["년도"] = rain["년도"].astype(int)

In [45]:
wind["월"] = wind["월"].astype(int)
wind["년도"] = wind["년도"].astype(int)

# 데이터 합치기

In [47]:
weather = pd.merge(temp,rain, left_index = True, right_index = True, how = "inner")

In [48]:
weather = pd.merge(weather, wind, left_index = True, right_index = True, how = "inner")

In [49]:
weather = weather.drop(["월_y","년도_y","월","년도","지역명_x","지역명_y"],axis =1)

In [50]:
weather #기온,강수량,평균풍속 데이터 합치기

,평균기온,월_x,년도_x,강수량,지역명,평균풍속
0,-0.6,1,2020,74.9,강원도,1.4
1,0.5,2,2020,50.1,강원도,1.7
2,5.8,3,2020,20.9,강원도,1.2
3,9.1,4,2020,27.2,강원도,1.6
4,16.6,5,2020,118.4,강원도,2.0
...,...,...,...,...,...,...
355,24.6,8,2022,425.3,충청북도,1.5
356,20.3,9,2022,100.1,충청북도,1.2
357,12.8,10,2022,88.3,충청북도,1.5
358,7.9,11,2022,55.0,충청북도,1.7


In [51]:
#컬럼명 재설정
weather = weather.rename(columns={'월_x':'월'})
weather = weather.rename(columns={'년도_x':'년도'})

In [52]:
new_columns = ["지역명","년도","월","평균기온","강수량","평균풍속"]
weather = weather[new_columns] #해당컬럼만 가지는 데이터프레임 새로 제작

In [53]:
weather.head()

,지역명,년도,월,평균기온,강수량,평균풍속
0,강원도,2020,1,-0.6,74.9,1.4
1,강원도,2020,2,0.5,50.1,1.7
2,강원도,2020,3,5.8,20.9,1.2
3,강원도,2020,4,9.1,27.2,1.6
4,강원도,2020,5,16.6,118.4,2.0


In [54]:
#기상청 데이터와 본데이터 합치기
data = pd.merge(data, weather, left_on = ["지역","사고발생연도","사고발생월"],
        right_on = ["지역명","년도","월"], left_index = False, right_index = False,
        how = "left")

In [55]:
#사고발생연도중 2018,2019년도 제거
data = data[~data["사고발생연도"].isin([2018,2019])]

#데터 인덱스 재설정
data = data.reset_index()

In [56]:
#불필요한 컬럼 제거
data = data.drop(["지역명","년도","월","index","사고발생일","학교급"],axis =1)

In [57]:
data

,지역,사고자구분,사고자성별,사고발생요일,사고시간,사고장소,사고부위,사고형태,사고매개물,사고당시활동,...,new_시간대,new_사고자학년,사고발생연도,사고발생월,사고발생일자,new_사고부위,new_학교급,평균기온,강수량,평균풍속
0,강원도,일반학생,남,목,등하교,운동장,다리,낙상-미끄러짐,자연(사람/동물/식물 등),보행/주행,...,아침,6,2020,1,2,하체,초,-0.6,74.9,1.4
1,강원도,일반학생,남,목,수업시간,교실,손,물리적힘 노출,"기계 도구류(기계선반, 재봉틀기계 등)",기타,...,아침,3,2020,1,2,상체,중,-0.6,74.9,1.4
2,강원도,일반학생,여,금,휴식시간 및 청소시간,통로,다리,낙상-넘어짐,건물(문/창문/바닥/벽 등),보행/주행,...,아침,3,2020,1,3,하체,중,-0.6,74.9,1.4
3,강원도,일반학생,남,목,수업시간,교실,손,물리적힘 노출,날카로운 물건(칼/가위/송곳 등),실험실습,...,낮,2,2020,1,2,상체,고,-0.6,74.9,1.4
4,강원도,일반학생,여,목,점심시간,운동장,다리,낙상-미끄러짐,기타,보행/주행,...,낮,2,2020,1,2,하체,초,-0.6,74.9,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279332,경기도,일반학생,남,목,체육수업,부속시설,발,낙상-넘어짐,자연(사람/동물/식물 등),구기운동,...,낮,2,2022,12,29,하체,고,-3.6,14.2,1.8
279333,경기도,일반학생,남,금,학교행사,운동장,손,물리적힘 노출,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),기타,...,낮,1,2022,12,30,상체,중,-3.6,14.2,1.8
279334,경기도,일반학생,남,금,점심시간,운동장,발,낙상-미끄러짐,건물(문/창문/바닥/벽 등),보행/주행,...,낮,1,2022,12,30,하체,중,-3.6,14.2,1.8
279335,경기도,일반학생,남,금,등하교,운동장,팔,낙상-미끄러짐,건물(문/창문/바닥/벽 등),보행/주행,...,아침,1,2022,12,30,상체,중,-3.6,14.2,1.8


# 데이터 저장하기

In [58]:
data.to_excel("data.xlsx", index = False)

# 데이터 불러오기

In [59]:
data = pd.read_excel("data.xlsx")

# 전국 기상청 데이터

In [60]:
data_rain = pd.read_csv("강수량.csv", skiprows =7,  encoding = "cp949") #전국 강수량 데이터 불러오기
data_temp = pd.read_csv("기온.csv", skiprows =7,  encoding = "cp949") #전국 기온 데이터 불러오기
data_wind = pd.read_csv("평균풍속.csv", skiprows =8,  encoding = "cp949") #전국 평균풍속 데이터 불러오기
data_wind = data_wind.drop(['\t\t지역번호'], axis =1) #평균풍속 데이터 불필요한 컬럼 제거

In [81]:
#년도 - 월 분리 중 년도 데이터
def remove_minute(input_string):
    input_string = input_string.split("-") #'-'을 기준으로 날짜 중 년도 데이터 저장
    modified_string = input_string[0]
    return modified_string

#년도 - 월 분리 중 년도 데이터
def remove_minute_(input_string):
    input_string = input_string.split("-") #'-'을 기준으로 날짜 중 필요한 부분부터 년도 데이터 저장
    modified_string = input_string[0][-4:]
    return modified_string


#년도 - 월 분리 중 월 데이터
def remove_minute1(input_string):
    input_string = input_string.split("-") #'-'을 기준으로 날짜 중 월 데이터 저장
    modified_string = input_string[1]
    return modified_string

In [62]:
#강수량 데이터에 '년도','사고발생월' 컬럼 추가
data_rain['년도'] = data_rain['년월'].apply(remove_minute)
data_rain["사고발생월"] = data_rain['년월'].apply(remove_minute1)

In [63]:
#기온 데이터에 '년도','사고발생월' 컬럼 추가
data_temp['년도'] = data_temp['년월'].apply(remove_minute_)
data_temp["사고발생월"] = data_temp['년월'].apply(remove_minute1)

In [64]:
#평균풍속 데이터에 '년도','사고발생월' 컬럼 추가
data_wind['년도'] = data_wind['일시'].apply(remove_minute)
data_wind['사고발생월'] = data_wind['일시'].apply(remove_minute1)

## drop  & rename

In [65]:
#불필요한 컬럼 제거
data_rain = data_rain.drop(["년월","지점"],axis =1)
data_temp = data_temp.drop(["년월","지점","평균최저기온(℃)","평균최고기온(℃)"], axis= 1)
data_wind = data_wind.drop(["지역명","일시","최대풍속(m/s)", "최대풍속지점", "최대순간풍속(m/s)", "최대순간풍속지점"], axis =1)

In [66]:
#컬럼명 재설정
data_rain = data_rain.rename(columns={'강수량(mm)':'전국 강수량'})
data_temp = data_temp.rename(columns={'평균기온(℃)':'전국 평균기온'})
data_wind = data_wind.rename(columns={'평균풍속(m/s)':'전국 평균풍속'})

In [67]:
#데이터 타입 바꾸기
data_rain = data_rain.astype('float')
data_temp = data_temp.astype('float')
data_wind = data_wind.astype('float')

In [68]:
#기온, 강수량, 평균풍속 데이터 합치기
data_wea = pd.merge(data_rain, data_temp, how = "outer")
data_weather = pd.merge(data_wea, data_wind, how= "outer")

In [69]:
data_weather

,전국 강수량,년도,사고발생월,전국 평균기온,전국 평균풍속
0,83.7,2020.0,1.0,2.5,1.7
1,58.7,2020.0,2.0,3.4,1.8
2,29.2,2020.0,3.0,7.7,2.1
3,41.0,2020.0,4.0,10.6,2.3
4,103.5,2020.0,5.0,17.6,1.9
5,182.9,2020.0,6.0,22.7,1.7
6,427.7,2020.0,7.0,22.5,1.5
7,420.8,2020.0,8.0,26.4,1.8
8,219.1,2020.0,9.0,20.1,1.8
9,10.5,2020.0,10.0,13.7,1.5


In [70]:
#본데이터와 전국 기상청 데이터 합치기
data = pd.merge(data, data_weather, left_on = ["사고발생연도","사고발생월"],
        right_on = ["년도","사고발생월"], left_index = False, right_index = False,
        how = "left")

In [71]:
#년도 컬럼 지우기
data = data.drop("년도",axis =1)

In [72]:
data

,지역,사고자구분,사고자성별,사고발생요일,사고시간,사고장소,사고부위,사고형태,사고매개물,사고당시활동,...,사고발생월,사고발생일자,new_사고부위,new_학교급,평균기온,강수량,평균풍속,전국 강수량,전국 평균기온,전국 평균풍속
0,강원도,일반학생,남,목,등하교,운동장,다리,낙상-미끄러짐,자연(사람/동물/식물 등),보행/주행,...,1,2,하체,초,-0.6,74.9,1.4,83.7,2.5,1.7
1,강원도,일반학생,남,목,수업시간,교실,손,물리적힘 노출,"기계 도구류(기계선반, 재봉틀기계 등)",기타,...,1,2,상체,중,-0.6,74.9,1.4,83.7,2.5,1.7
2,강원도,일반학생,여,금,휴식시간 및 청소시간,통로,다리,낙상-넘어짐,건물(문/창문/바닥/벽 등),보행/주행,...,1,3,하체,중,-0.6,74.9,1.4,83.7,2.5,1.7
3,강원도,일반학생,남,목,수업시간,교실,손,물리적힘 노출,날카로운 물건(칼/가위/송곳 등),실험실습,...,1,2,상체,고,-0.6,74.9,1.4,83.7,2.5,1.7
4,강원도,일반학생,여,목,점심시간,운동장,다리,낙상-미끄러짐,기타,보행/주행,...,1,2,하체,초,-0.6,74.9,1.4,83.7,2.5,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279332,경기도,일반학생,남,목,체육수업,부속시설,발,낙상-넘어짐,자연(사람/동물/식물 등),구기운동,...,12,29,하체,고,-3.6,14.2,1.8,15.2,-1.4,1.8
279333,경기도,일반학생,남,금,학교행사,운동장,손,물리적힘 노출,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),기타,...,12,30,상체,중,-3.6,14.2,1.8,15.2,-1.4,1.8
279334,경기도,일반학생,남,금,점심시간,운동장,발,낙상-미끄러짐,건물(문/창문/바닥/벽 등),보행/주행,...,12,30,하체,중,-3.6,14.2,1.8,15.2,-1.4,1.8
279335,경기도,일반학생,남,금,등하교,운동장,팔,낙상-미끄러짐,건물(문/창문/바닥/벽 등),보행/주행,...,12,30,상체,중,-3.6,14.2,1.8,15.2,-1.4,1.8


# 최종 데이터 저장

In [73]:
data.to_excel("final_data.xlsx", index = False) #데이터 프레임 xlsx 파일로 저장

In [74]:
data_2020 = data[data["사고발생연도"] ==2020] #사고발생연도가 2020년인 데이터

In [75]:
data_2020.to_excel("data_2020.xlsx", index = False) #데이터 프레임 xlsx 파일로 저장

In [76]:
data_2021 = data[data["사고발생연도"] ==2021] #사고발생연도가 2021년인 데이터

In [77]:
data_2021.to_excel("data_2021.xlsx", index = False) #데이터 프레임 xlsx 파일로 저장

In [78]:
data_2022 = data[data["사고발생연도"] ==2022] #사고발생연도가 2022년인 데이터

In [79]:
data_2022.to_excel("data_2022.xlsx", index = False) #데이터 프레임 xlsx 파일로 저장